## Quantum Harmonic Oscillator with Time Dependent Potential

In [4]:
import numpy as np
from qhm_helper import momentum_operator 
def potential_operator_t(x, t, m, w, A=1, alpha=0.1):
    return 0.5 * m * w**2 * x**2 + A * np.sin(alpha * t) * x

def kinetic_operator(psi, dx, m):
    return (1/(2 *m)) * momentum_operator(momentum_operator(psi, dx), dx)

def hamiltonian_operator(psi, x, t, dx, m, w):
    return kinetic_operator(psi, dx, m) + potential_operator_t(x, t, m, w) * psi

def d_psi(psi, x, t, dx, m, w):
    return -1j * hamiltonian_operator(psi, x, t, dx, m, w)

def runge_kutta(psi, x, t, dx, m, w, dt):
    k1 = d_psi(psi, x, t, dx, m, w)
    k2 = d_psi(psi + 0.5 * dt * k1, x, t + 0.5 * dt, dx, m, w)
    k3 = d_psi(psi + 0.5 * dt * k2, x, t + 0.5 * dt, dx, m, w)
    k4 = d_psi(psi + dt * k3, x, t + dt, dx, m, w)
    psi = psi + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    # Normalize
    psi /= np.sqrt(np.sum(np.abs(psi)**2) * dx)
    return psi

In [5]:
import numpy as np   
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from qhm_helper import momentum_operator, energy, potential_energy, kinetic_energy, position_expectation, position_variance, rk_step

dx = 0.02
x_min = -10
x_max = 10
x = np.arange(x_min, x_max, dx)
print(f"len(x): {len(x)}")
print(f"dx check: {x[1] - x[0]}")
m = 2.5e2
w = 4e-2
print(f"mw^2: {m*w**2}")
print(f"1/m: {1/m}")

# KE_constant = 0.00001
# PE_constant = 0.1

dt = 0.01
steps = 100000
psi_x = np.exp(0.5 * m * w * -((x-1)**2))
# psi_x = np.exp(0.5 * -((x-1)**2))
psi_x = psi_x / np.sqrt(np.sum(np.abs(psi_x)**2) * dx)

len(x): 1000
dx check: 0.019999999999999574
mw^2: 0.4
1/m: 0.004


In [6]:
step_frequency = 1000
psi_x_t = np.zeros(shape=(step_frequency, len(psi_x)), dtype=complex)
V_x_t = np.zeros(shape=(step_frequency, len(psi_x)))
position_expectations = np.zeros(shape=(step_frequency))
position_variances = np.zeros(shape=(step_frequency))
kinetic_energy_t = np.zeros(shape=(step_frequency))
potential_energy_t = np.zeros(shape=(step_frequency))
energy_t = np.zeros(shape=(step_frequency))
t = 0
for i in range(steps):
    psi_x = runge_kutta(psi_x, x, t, dx, m, w, dt)

    if i % int(steps/step_frequency) == 0:
        print(f"i: {i}")
        # record psi_x
        psi_x_t[int(i/int(steps/step_frequency))] = psi_x
        V_x_t[int(i/int(steps/step_frequency))] = potential_operator_t(x, t, m, w)
        position_expectations[int(i/int(steps/step_frequency))] = np.real(position_expectation(psi_x, x, dx))
        energy_t[int(i/int(steps/step_frequency))] = energy(psi_x, x, dx, m, w)
        print(f"wave function norm: {np.sum(np.abs(psi_x)**2) * dx}")
        print(f"Energy: {energy_t[int(i/int(steps/step_frequency))]}")
        print(f"Position Expectation: {position_expectations[int(i/int(steps/step_frequency))]}")
        np.save("data/qhm_t/psi_x_t_progress.npy", psi_x_t)
        np.save("data/qhm_t/V_x_t_progress.npy", V_x_t)

    E = energy(psi_x, dx, m, w)
    if np.real(E) > 5:
        break

    t += dt


i: 0
wave function norm: 1.0000000000000002
Energy: 0.21998002669319117
Position Expectation: 0.9999999202529617


<ipython-input-6-1089070079bb>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  energy_t[int(i/int(steps/step_frequency))] = energy(psi_x, x, dx, m, w)


i: 100
wave function norm: 1.0000000000000002
Energy: 0.220039863022555
Position Expectation: 0.9991189394469144
i: 200
wave function norm: 1.0000000000000002
Energy: 0.22049001301510768
Position Expectation: 0.9962450795411785
i: 300
wave function norm: 1.0000000000000002
Energy: 0.22181510238786467
Position Expectation: 0.9909912902976055
i: 400
wave function norm: 1.0
Energy: 0.22458983456913587
Position Expectation: 0.9829847839115283
i: 500
wave function norm: 1.0000000000000002
Energy: 0.22945555497497772
Position Expectation: 0.9718731186253464
i: 600
wave function norm: 1.0000000000000002
Energy: 0.2370914686243393
Position Expectation: 0.9573304765576562
i: 700
wave function norm: 1.0000000000000002
Energy: 0.24818160658088168
Position Expectation: 0.9390639657519766
i: 800
wave function norm: 1.0000000000000002
Energy: 0.263379003737036
Position Expectation: 0.9168197208340059
i: 900
wave function norm: 1.0000000000000002
Energy: 0.28326887612507573
Position Expectation: 0.89